In [1]:
import pandas as pd

from actors import (Ship, Geopoint, speed_limitations, detect_point_category, IceCategory,
                    VesselCategory, Port, Geopoint, VesselMoveStatus, Icebreaker, Route)
from utils import find_and_serialize_neighbors, load_serialized_neighbors
from route_metrics import ice_metrics_on_route, get_closest_grid_points_on_route_step, route_steps_on_edge, ice_integral_coefficient_on_step

grid = pd.read_excel(r"data\velocity.xlsx")
routes = pd.read_excel(r"data\routes.xlsx")
ports = pd.read_excel(r"data\ports.xlsx")
vessels = pd.read_excel(r"data\vessels.xlsx")
iceboats = pd.read_excel(r"data\iceboat.xlsx")
schedule = pd.read_excel(r"data\schedule.xlsx")

In [2]:
from actors import ShipTimed, IceBreaker, Geopoint, VesselMoveStatus
time_tick = 1 / 6 # 10 min

def make_ship(config: dict):
    if '_' in config['iceclass']:
        start_geo = routes.iloc[:, 3:].sample(1).values[0][:2]
        return IceBreaker(
            name=config.get('name'),
            category=VesselCategory.arc92,
            location_point=Geopoint(start_geo[0], start_geo[1]),
            route_request=None, # им никуда не надо
            status=VesselMoveStatus.waiting,
            max_speed=18.50,
            avg_speed=0.,
            curr_speed=0.,
            tick=time_tick,
        )
    start_geo = routes.iloc[:, 3:].sample(1).values[0]
    return ShipTimed(
        name=config.get('name'),
        category=VesselCategory.arc6,
        location_point=Geopoint(start_geo[0], start_geo[1]),
        route_request=Geopoint(start_geo[2], start_geo[3]),
        status=VesselMoveStatus.waiting,
        max_speed=19,
        avg_speed=0.,
        curr_speed=0.,
        tick=time_tick,
    )

In [3]:
time_tick = 1 / 6 # 10 min
#ships icebrteakers
ships_list = [make_ship(ship_stat) for _, ship_stat in vessels.to_dict(orient='index').items()]
# grid
find_and_serialize_neighbors(grid=grid[["lat", "lon"]].values, static_folder="data")

config = {
    'max_episode_steps': 200,
    'neighbors_shape': (1, 32, ),
    'serialized_neighbors': load_serialized_neighbors("data"),
    'grid': grid,
    #'routes': routes.iloc[:, 3:],
    'date_start': 2,
    'time_tick': time_tick,
    'ice_breaker_counts': 4,
    'ships_count': 42,
    'ships_list': ships_list
}

In [4]:
from envs.waterworld_multiagent import WaterWorldMultiEnv
import warnings
warnings.filterwarnings("ignore")

wwme = WaterWorldMultiEnv(config)
w, _ = wwme.reset()

d:\Code\Python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-15 04:30:20,944	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-06-15 04:30:23,622	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [8]:
from ray.rllib.algorithms import PPOConfig


try:
    algo.stop()
except:
    pass

training_options = {
    'train_batch_size': 128,
    #'sgd_minibatch': 64,
    #'lr': 1e-05,
    #'num_sgd_iter': 22,
    'model': {
        #'fcnet_activation': 'relu',
        #'fcnet_hiddens': [256, 256],
        'vf_share_layers': False,
        #'use_lstm': True,
    }
}

cql_config = (
    PPOConfig()
    .environment(env=WaterWorldMultiEnv, env_config=config)
    .framework('torch')
    .rollouts(num_rollout_workers=0, rollout_fragment_length=16)
    #.evaluation(evaluation_num_workers=0, evaluation_interval=1)
    .training(**training_options)
  
)

2024-06-15 04:32:44,061	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.env_runners(num_rollout_workers)` has been deprecated. Use `AlgorithmConfig.env_runners(num_env_runners)` instead. This will raise an error in the future!


In [9]:
algo = cql_config.build()

2024-06-15 04:32:46,109	WARNING util.py:61 -- Install gputil for GPU system monitoring.


In [10]:
algo.step()

{'custom_metrics': {},
 'episode_media': {},
 'info': {'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0,
     'grad_gnorm': 13.890784894556239,
     'cur_kl_coeff': 0.19999999999999996,
     'cur_lr': 5e-05,
     'total_loss': 1.1512150293594037,
     'policy_loss': -0.03031362803126483,
     'vf_loss': 1.1781983971271826,
     'vf_explained_var': 0.02687043659928916,
     'kl': 0.016651330070642418,
     'entropy': 1.9290467572384986,
     'entropy_coeff': 0.0},
    'model': {},
    'custom_metrics': {},
    'num_agent_steps_trained': 128.0,
    'num_grad_updates_lifetime': 690.5,
    'diff_num_grad_updates_vs_sampler_policy': 689.5}},
  'num_env_steps_sampled': 128,
  'num_env_steps_trained': 128,
  'num_agent_steps_sampled': 5888,
  'num_agent_steps_trained': 5888},
 'sampler_results': {'episode_reward_max': -506.8863479885392,
  'episode_reward_min': -2523.842856103267,
  'episode_reward_mean': -908.2910143504773,
  'episode_len_mean': 2.1333333333333333,
 

In [12]:
wwme.observation_space_contains({0: wwme.observation_space_sample()[0]})

True

In [11]:
wwme.observation_space_sample()[0]

OrderedDict([('angle', array([-0.7683665], dtype=float32)),
             ('compas', array([ 0.8292413 , -0.18910442], dtype=float32)),
             ('cordes', array([13.145371, 72.54387 ], dtype=float32)),
             ('ice_arc', 1),
             ('ice_breaker_cords',
              array([17.858513, 27.296387, 93.56423 , 79.19787 , 87.28299 , 28.966595,
                     78.04393 ,  9.793797], dtype=float32)),
             ('slide_circle',
              array([[ 7.716908 , 22.65159  ,  2.252321 , 17.597994 , 12.30989  ,
                      22.237587 , 19.089342 ,  4.6556683, 17.688524 , 13.798295 ,
                       8.2795725, 16.636795 ,  9.894301 , 16.12021  ,  6.3718944,
                       2.122717 , 15.771759 , 15.41072  , 23.243626 , 10.855424 ,
                       5.2255526, 21.661629 , 17.100037 , 11.671296 , 16.025246 ,
                       0.8830905, 20.369486 , 17.314754 , 15.16412  ,  3.582159 ,
                       1.19756  , 16.020609 ]], dtype=float3

In [14]:
d = [wwme.step(wwme.action_space_sample()) for _ in range(500)][-1]

In [19]:
d[3]

{0: False,
 1: False,
 2: True,
 3: False,
 4: False,
 6: True,
 7: True,
 8: True,
 9: False,
 12: True,
 14: True,
 15: False,
 16: True,
 17: True,
 18: True,
 19: True,
 22: True,
 23: True,
 24: True,
 25: True,
 26: True,
 27: True,
 28: True,
 29: True,
 31: True,
 32: True,
 33: True,
 35: False,
 36: False,
 37: False,
 38: True,
 39: True,
 41: True,
 42: True,
 43: True,
 '__all__': True,
 11: True,
 13: False,
 20: False,
 30: False,
 40: False,
 45: False,
 5: True,
 10: True,
 34: True,
 21: True,
 44: False}

In [10]:
ccc = [len(wwme.step(wwme.action_space_sample())[0].keys()) for i in range(500)]

In [12]:
min(ccc)

46

In [29]:
wwme.reset()

({42: {'angle': array([0.], dtype=float32),
   'compas': array([0., 0.], dtype=float32),
   'slide_circle': array([[10.967523 , 10.967523 , 12.276084 , 12.276084 , 12.276084 ,
           12.276084 , 12.276084 , 12.276084 , 12.276084 , 12.276084 ,
           20.479004 , 15.674437 , 12.276084 , 12.276084 , 12.276084 ,
           20.479004 , 12.276084 , 12.276084 , 12.276084 , 12.539749 ,
           13.047549 , 20.479004 , 13.047549 , 20.518064 ,  9.4636545,
           12.276084 , 13.047549 , 12.276084 , 12.276084 , 12.539749 ,
           13.047549 , 20.918446 ]], dtype=float32),
   'velocity': array([13.875], dtype=float32),
   'ice_arc': array([10.], dtype=float32),
   'cordes': array([ 75.9, 152.6], dtype=float32),
   'ice_breaker_cords': array([ 75.9, 152.6,  70.3,  57.8,  75.5, 131.5,  77.8, 104.1],
         dtype=float32)},
  43: {'angle': array([0.], dtype=float32),
   'compas': array([0., 0.], dtype=float32),
   'slide_circle': array([[21.855923, 21.904749, 21.855923, 20.039562, 2

In [19]:
ships_list[10]

ShipTimed(name='ИНЖЕНЕР ВЕШНЯКОВ', category=<VesselCategory.arc6: 6>, location_point=Geopoint(latitude=64.2, longitude=185.98333333333332), route_request=Geopoint(latitude=60.7, longitude=175.3), status=<VesselMoveStatus.waiting: 0>, max_speed=19, avg_speed=0.0, curr_speed=13.299999999999999, total_time=0.3333333333333333, tick=0.16666666666666666, _last_x=0, _last_y=-1)

In [23]:
wwme.envs[10]._is_trunc()

True

In [27]:
wwme.envs[10].ship.location_point.longitude < -200

False

In [34]:
wwme.envs[10].reset(options={'ship': wwme.envs[10].ship})

({'angle': array([0.], dtype=float32),
  'compas': array([0.2467685 , 0.75323147], dtype=float32),
  'slide_circle': array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
        dtype=float32),
  'velocity': array([13.3], dtype=float32),
  'ice_arc': array([6.], dtype=float32)},
 {})

In [36]:
wwme.envs[10]._last_slide_circle

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])